## Setup

In [1]:
import os

from Db import DataBase
from Category import Category
from Feature import Feature

from Trainset import Trainset
from Testset import Testset
from Trainer import Trainer

## Load Data

In [2]:
# Load Data
file_path = os.getcwd() + "/../data/"
dbase = DataBase(file_path)

## Data Preprocessing

In [3]:
# orignal_price
original_price = dbase.original_price()

# baskets data
baskets_data = dbase.load_basket_data()

In [4]:
# Product Category Table
catClass = Category(baskets_data)
prods_cat_table = catClass.generate_product_category_table()

In [5]:
print(prods_cat_table.shape)
# prods_cat_table.tail(60)

(250, 2)


In [6]:
# Split Data
baskets_train, baskets_test, coupons_train, coupons_test = dbase.generate_split_data_with_category(prods_cat_table)

print(baskets_train.head(2))
print(baskets_test.head(2))
print(coupons_train.head(2))
print(coupons_test.head(2))

/Users/jungwoojamesjeong/opt/anaconda3/envs/hu-bads/lib/python3.8/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


   week shopper product  price  target category
0     0       0      71    629       1        3
1     0       0      91    605       1        5
   week shopper  product  price  target category
0    89       0       67    637       1       21
1    89       0       71    629       1        3
   week shopper product  discount category
0     0       0      35        35        7
1     0       0     193        40       19
   week shopper  product  discount category
0    89       0      131        30       23
1    89       0       16        25       17


## Feature Engineering

In [7]:
# Define feat
print("feature engineering begin")
feat = Feature(baskets_train, coupons_train)

feature engineering begin


In [8]:
total_count_of_product = feat.total_count_of_product()
reordered_category = (baskets_train.groupby(['shopper']).category.value_counts()>1).astype(int)
reordered_product = feat.reordered_product()
category_count = feat.category_count()
reordered_category = feat.reordered_category()
coupon_in_same_category = feat.coupon_in_same_category()
average_price_per_shopper = feat.average_price_per_shopper()
average_basket_size = feat.average_basket_size()
unique_products_per_shopper = feat.unique_products_per_shopper()
unique_categories_per_shopper = feat.unique_categories_per_shopper()

In [9]:
# ratio_of_reordered_products_per_shopper = feat.ratio_of_reordered_products_per_shopper()
# ratio_of_reordered_categories_per_shopper = feat.ratio_of_reordered_categories_per_shopper()


In [10]:
print("feature engineering finished")

feature engineering finished


## Training Set

In [11]:
# Train Table
train_t = Trainset(baskets_train, coupons_train, original_price)

In [12]:
## featureless_training_set
featureless_training_set = train_t.generate_featureless_training_set(prods_cat_table)
print("----- featureless training set -----")
print(featureless_training_set.head(2))

ValueError: maximum supported dimension for an ndarray is 32, found 250

In [ ]:
## Generate training set
training_set = train_t.populate_features(
    featureless_training_set,
    original_price,
    total_count_of_product,
    reordered_product,
    category_count,
    reordered_category,
    coupon_in_same_category,
    average_price_per_shopper,
    average_basket_size,
    unique_products_per_shopper,
    unique_categories_per_shopper
#     ratio_of_reordered_products_per_shopper,
#     ratio_of_reordered_categories_per_shopper,
)
print("===== training_set =====")
print(training_set.head(2))

In [ ]:
# X_train, y_train
X_train, y_train = train_t.split_trainingset_to_X_train_and_y_train(training_set)
print(X_train.head(2))
print(y_train.head(2))

## Testing Set

In [ ]:
# testing table
test_t = Testset(baskets_test, coupons_test)

In [ ]:
# full_df_test
full_df_test = test_t.generate_full_df_test()

In [ ]:
# Generate testing set
testing_set = test_t.generate_training_set(
    prods_cat_table,
    original_price,
    total_count_of_product,
    reordered_product,
    category_count,
    reordered_category,
    coupon_in_same_category,
    average_price_per_shopper,
    average_basket_size,
    unique_products_per_shopper,
    unique_categories_per_shopper,
    training_set
#     ratio_of_reordered_products_per_shopper,
#     ratio_of_reordered_categories_per_shopper,
)

In [ ]:
X_test, y_test = test_t.split_testingset_to_X_test_and_y_test(testing_set)
print(X_test.head(2))
print(y_test.head(2))

## Trainer

In [ ]:
trainer = Trainer(X_train, y_train, X_test, y_test)

In [ ]:
model = trainer.fit_model()